In [ ]:
from pathlib import Path
import pandas as pd
import altair as alt
from scipy.stats.mstats import gmean
import numpy as np

In [ ]:
# Combinations of width configuration mode and partial clock gating on/off.

modes = {'nofuse':'Original', 'nofusepenalty':'Packing', 'fusepenalty':'Fusing'}
power_modes = {'nofuse':'power', 'nofusepenalty':'power_clkgated', 'fusepenalty':'power_clkgated'}
modes_order = {'Original':0, 'Packing':1, 'Fusing':2}

In [ ]:
# Benchmarks to include, and their name in the plot.

benchmarks = {
    #'hmmer-small_1':'hmmer',
    #'fft_simsmall_1':'fft-splash',
    'integerNN':'integerNN',
    'streamvbyte':'streamvbyte',
    'img_cartoon_image1':'cartoon',
    'img_cartoon_image2':'cartoon',
    'img_cartoon_image3':'cartoon',
    'img_cartoon_image4':'cartoon',
    'img_cartoon_image5':'cartoon',
    'img_canny_image1':'canny',
    'img_canny_image2':'canny',
    'img_canny_image3':'canny',
    'img_canny_image4':'canny',
    'img_canny_image5':'canny',
    'img_hist_image1':'hist',
    'img_hist_image2':'hist',
    'img_hist_image3':'hist',
    'img_hist_image4':'hist',
    'img_hist_image5':'hist',
    'img_integral_image1':'img_integral',
    'img_integral_image2':'img_integral',
    'img_integral_image3':'img_integral',
    'img_integral_image4':'img_integral',
    'img_integral_image5':'img_integral',
    'conv_image1':'conv',
    'conv_image2':'conv',
    'conv_image3':'conv',
    'conv_image4':'conv',
    'conv_image5':'conv',
    'img_erode_image1':'erode',
    'img_erode_image2':'erode',
    'img_erode_image3':'erode',
    'img_erode_image4':'erode',
    'img_erode_image5':'erode',
    'img_median_image1':'median',
    'img_median_image2':'median',
    'img_median_image3':'median',
    'img_median_image4':'median',
    'img_median_image5':'median',
    'amax_cols_normal_16bit_0':'amax',
    'amax_cols_normal_16bit_1':'amax',
    'amax_cols_normal_16bit_2':'amax',
    'amax_cols_normal_16bit_3':'amax',
    'amax_cols_normal_16bit_4':'amax',
    #'asum_cols_normal_16bit_0':'asum',
    #'asum_cols_normal_16bit_1':'asum',
    #'asum_cols_normal_16bit_2':'asum',
    #'asum_cols_normal_16bit_3':'asum',
    #'asum_cols_normal_16bit_4':'asum',
    #'gemv_normal_16bit_0':'gemv',
    #'gemv_normal_16bit_1':'gemv',
    #'gemv_normal_16bit_2':'gemv',
    #'gemv_normal_16bit_3':'gemv',
    #'gemv_normal_16bit_4':'gemv',
    #'ger_normal_16bit_0':'ger',
    #'ger_normal_16bit_1':'ger',
    #'ger_normal_16bit_2':'ger',
    #'ger_normal_16bit_3':'ger',
    #'ger_normal_16bit_4':'ger',
    'sqnrm2_cols_normal_16bit_0':'sqnrm2',
    'sqnrm2_cols_normal_16bit_1':'sqnrm2',
    'sqnrm2_cols_normal_16bit_2':'sqnrm2',
    'sqnrm2_cols_normal_16bit_3':'sqnrm2',
    'sqnrm2_cols_normal_16bit_4':'sqnrm2',
    'gemm_normal_16bit_0':'gemm',
    'gemm_normal_16bit_1':'gemm',
    'gemm_normal_16bit_2':'gemm',
    'gemm_normal_16bit_3':'gemm',
    'gemm_normal_16bit_4':'gemm',
    'fft_normal_8bit_0':'fft',
    'fft_normal_8bit_1':'fft',
    'fft_normal_8bit_2':'fft',
    'fft_normal_8bit_3':'fft',
    'fft_normal_8bit_4':'fft',
}

bench_order = ['hmmer', 'streamvbyte', 'integerNN', 'cartoon', 'sqnrm2', 'amax', 'gemm', 'fft', 'conv', 'median', 'img_integral', 'hist', 'canny', 'erode']

In [ ]:
# Reference config for each CPU.

cpus     = ['A76', 'HP']
ref_mode = 'Original'
ref_fu   = {'A76':2, 'LP':2, 'HP':3}

In [ ]:
# Parameter filtering.

component = 'fp/simd alu'

In [ ]:
# Plot labels.

labels = {'Dynamic':'Dynamic Energy Reduction [%]',
          'Leakage':'Leakage Energy Reduction [%]',
          'Total':'Total Energy Reduction [%]'}

In [ ]:
# List all the run output folder paths.

folder = './stats'

runs = list(Path(folder).glob('**/roi.txt'))
runs = [r.parents[0] for r in runs]
#runs

In [ ]:
# Create table with the energy and time results for the specified runs.

df = []

for r in runs:
    # Params that can only be obtained from the path.
    cpu = r.parents[1].name.split('-')[0]
    mode = r.parents[1].name.split('-')[2]
    
    # Params that can be obtained from config.csv.
    try:
        param = pd.read_csv(r/'config.csv', header=0)
    except:
        continue
    fu = param.iloc[0]['simdCount']
    wBlock = int(param.iloc[0]['widthBlockSize'])
    bench = Path(param.iloc[0]['bootscript']).stem
    
    if cpu in cpus and mode in modes.keys() and bench in benchmarks.keys():
        # Time results.
        time = pd.read_csv(r/'time.csv', header=0)
        try:
            time = float(time.iloc[0]['sim seconds'])
        except TypeError:
            continue
        except IndexError:
            continue

        # Mcpat results. (FIXME: Can change here for more components!)
        mcpat = pd.read_csv(r/'{}.csv'.format(power_modes[mode]))
        mcpat = mcpat[mcpat['Component'] == component]
        
        dynamic = float(mcpat.loc[:, 'Runtime Dynamic [W]'].iloc[0] * time)
        leakage = float((mcpat.loc[:, ['Gate Leakage [W]', 'Subthreshold Leakage [W]']].sum(axis='columns')).iloc[0]) * time

        # Add entry to the table.
        df.append({'Benchmark':bench, 'CPU':cpu, 'SIMD FU':fu, 'Mode':modes[mode], 'Width Block':wBlock,
                   'Time':time, 'Leakage':leakage, 'Dynamic':dynamic, 'Total':dynamic+leakage})
    
df = pd.DataFrame(df)
#df

In [ ]:
# Normalize time (slowdown) and energy in comparison to the reference config for that CPU.

# Calculate normalized metrics for each run.
df_norm = df.copy()
df_norm['Ref SIMD FU'] = df_norm['CPU'].replace(ref_fu)
df_norm['Ref Mode'] = ref_mode
df_norm = df_norm.merge(df, left_on=['Benchmark', 'CPU', 'Width Block', 'Ref SIMD FU', 'Ref Mode'],
                        right_on=['Benchmark', 'CPU', 'Width Block', 'SIMD FU', 'Mode'],
                        suffixes=('', ' ref'))
df_norm['Slow Down'] = df_norm['Time'] / df_norm['Time ref']
df_norm['Dynamic']   = df_norm['Dynamic'] / df_norm['Dynamic ref']
df_norm['Leakage']   = df_norm['Leakage'] / df_norm['Leakage ref']
df_norm['Total']     = df_norm['Total'] / df_norm['Total ref']
df_norm = df_norm.drop(columns=['Time', 'Ref SIMD FU', 'Ref Mode', 'SIMD FU ref', 'Mode ref', 'Time ref',
                                'Leakage ref', 'Dynamic ref', 'Total ref'])
df_norm = pd.melt(df_norm, id_vars=['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block'],
                  var_name='Metric', value_name='Value').copy()
df_norm = df_norm.set_index(['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'])

# Apply geometric mean to reduce several runs of the same benchmark.
df_norm = df_norm.groupby(by=[benchmarks, None, None, None, None, None],
                          level=['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'])['Value'].apply(lambda group: gmean(group).mean()).reset_index().set_index(['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'])

# Create 'average' benchmark using the geometric mean.
average = df_norm.groupby(['CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'], as_index=True, sort=False)['Value'].apply(lambda group: gmean(group).mean()).reset_index()
average['Benchmark'] = 'average'
average = average.set_index(['Benchmark', 'CPU', 'SIMD FU', 'Mode', 'Width Block', 'Metric'])
df_norm = pd.concat([df_norm, average])


# Calculate reduction % for energy metrics.  
df_norm.loc(axis=0)[pd.IndexSlice[:,:,:,:,:,['Dynamic', 'Leakage', 'Total']]] = \
    100 * (1 - df_norm.loc(axis=0)[pd.IndexSlice[:,:,:,:,:,['Dynamic', 'Leakage', 'Total']]])

df_norm

In [ ]:
# Help function for plots.

def filter_data(df, bench=None, cpu=None, metric=None, wBlock=None, fu=None, mode=None):
    if bench is not None:
        df = df[df['Benchmark'].isin(bench)]
    if cpu is not None:
        df = df[df['CPU'].isin(cpu)]
    if metric is not None:
        df = df[df['Metric'].isin(metric)]
    if wBlock is not None:
        df = df[df['Width Block'].isin(wBlock)]
    if fu is not None:
        if type(fu) is list:
            df = df[df['SIMD FU'].isin(fu)]
        elif type(fu) is dict:
            df = df[df['SIMD FU'] == df['CPU'].replace(fu)]
    if mode is not None:
        df = df[df['Mode'].isin(mode)]
    
    return df


def get_chart(df, bench=None, cpu=None, metric=None, wBlock=None, fu=None, mode=None, scheme='category10'):
    df_metric = filter_data(df, bench=bench, cpu=cpu, metric=metric, wBlock=wBlock, fu=fu, mode=mode)
        
    df_metric['Metric'] = df_metric['Metric'].replace(labels)
    
    ct = alt.Chart(df_metric).mark_line(point=True).encode(
        x=alt.X(
            'Benchmark:N',
            title="",
            axis=alt.Axis(labelAngle=-30),
            sort=bench_order
        ),
        y=alt.Y(
            'Value:Q',
            scale=alt.Scale(zero=False),
            title=""
        ),
        color=alt.Color(
            'Configuration:N',
            scale=alt.Scale(range=scheme),
            sort=config_order
        ),
        row=alt.Row(
            'Metric:N',
            title=""
        )
    ).properties(
        height=240,
        width=800
    )
    
    return ct


def configure_chart(ct):
    ct = ct.configure_axis(
        labelFontSize=15,
        titleFontSize=16,
        grid=True
    ).configure_header(
        labelFontSize=18,
        titleFontSize=18
    ).configure_legend(
        orient='right',
        titleFontSize=18,
        labelFontSize=16,
        symbolType='stroke',
        symbolSize=800,
        symbolStrokeWidth=5
    ).configure_line(
        size=2.5
    ).configure_point(
        size=75
    )
    return ct

In [ ]:
# Add config column.
df_plot = df_norm.reset_index().copy()
df_plot['Configuration'] = df_plot['CPU'] + '-' + df_plot['Mode'] + '-' + df_plot['SIMD FU'].astype(str) + 'FU'
df_plot['Config Order'] = df_plot['Mode'].replace(modes_order) * df_plot['SIMD FU'].max() + (df_plot['SIMD FU'].max()-df_plot['SIMD FU'])
df_plot

In [ ]:
# Get configuration order.
config_order = list(df_plot.sort_values(['CPU', 'Config Order'])['Configuration'].unique())
config_order

In [ ]:
# Plots A76.

scheme = ['#ffaa73', '#a84400',
          '#95baff', '#005dff',
          '#84cd84', '#007700']
ct_A76 = get_chart(df_plot, cpu=['A76'], metric=['Dynamic'], wBlock=[8], scheme=scheme) & \
         get_chart(df_plot, cpu=['A76'], metric=['Leakage'], wBlock=[8], scheme=scheme) & \
         get_chart(df_plot, cpu=['A76'], metric=['Total'], wBlock=[8], scheme=scheme) & \
         get_chart(df_plot, cpu=['A76'], metric=['Slow Down'], wBlock=[8], scheme=scheme)
ct_A76 = configure_chart(ct_A76)
ct_A76

In [ ]:
# Save A76 plot.

ct_A76.save('plots/a76_diffenergy_slowdown.svg', webdriver='firefox')

In [ ]:
# Plots HP.

scheme = ['#ffc9a5', '#ff8533', '#8c3800',
          '#c5d2e9', '#5ea1ff', '#004cd2',
          '#9ae59a', '#3abf3a', '#006000']
ct_HP = get_chart(df_plot, cpu=['HP'], metric=['Dynamic'], wBlock=[8], scheme=scheme) & \
        get_chart(df_plot, cpu=['HP'], metric=['Leakage'], wBlock=[8], scheme=scheme) & \
        get_chart(df_plot, cpu=['HP'], metric=['Total'], wBlock=[8], scheme=scheme) & \
        get_chart(df_plot, cpu=['HP'], metric=['Slow Down'], wBlock=[8], scheme=scheme)
ct_HP = configure_chart(ct_HP)
ct_HP

In [ ]:
# Save HP plot.

ct_HP.save('plots/hp_diffenergy_slowdown.svg', webdriver='firefox')

In [ ]:
# Plots latency penalty impact.

scheme = ['#95baff', '#005dff',
          '#84cd84', '#007700']
ct_penalty = get_chart(df_plot, cpu=['A76', 'HP'], metric=['Slow Down'], mode=['Packing', 'Fusing'], wBlock=[8], fu=ref_fu, scheme=scheme)
ct_penalty = configure_chart(ct_penalty)
ct_penalty

In [ ]:
# Save latency penalty plot.

ct_HP.save('plots/slowdown_penalty.svg', webdriver='firefox')